In [1]:
import numpy as np
import os, sys
import random

### Read in training and testing data GANwriting

Per wid to ensure that all writers are present in both datasets

In [2]:
max_word_len = 0
with open("gan.iam.tr_va.gt.filter27", 'r') as f_tr:
    data_tr = f_tr.readlines()
    data_tr = [i.strip().split(' ') for i in data_tr]
    tr_dict = dict()
    for i in data_tr:
        wid = i[0].split(',')[0]
        if len(i[1]) > max_word_len:
            max_word_len = len(i[1])
        if wid not in tr_dict.keys():
            tr_dict[wid] = [i]
        else:
            tr_dict[wid].append(i)

In [3]:
with open("gan.iam.test.gt.filter27", 'r') as f_te:
    data_te = f_te.readlines()
    data_te = [i.strip().split(' ') for i in data_te]
    te_dict = dict()
    for i in data_te:
        wid = i[0].split(',')[0]
        if len(i[1]) > max_word_len:
            max_word_len = len(i[1])
        if wid not in te_dict.keys():
            te_dict[wid] = [i]
        else:
            te_dict[wid].append(i)

In [4]:
max_word_len

7

The GANwriting test set contains only unseen writer styles. To train the GAN in the same way the paper did, we will want to maintain this later on

In [5]:
tr_wids = [key for key in tr_dict.keys()]
te_wids = [key for key in te_dict.keys()]

for wid in te_wids:
    if wid in tr_wids:
        print(wid)

In [6]:
print(tr_dict['049'][0])

['049,a03-034-00-00', 'Members']


Assigning half to gan dataset, and half to htr dataset for each wid. This also maintains class proportions between halves. 

The files are saved in different formats for reading in.

GAN: '049,a03-034-00-00 Members'
HTR: '049, a03/a03-034/a03-034-00-00.png Members'

In [7]:
gan_tr_f = "gan.iam.gan_tr_va.filter27"
htr_f = "htr.iam.all.filter27"

f_gan = open(gan_tr_f, 'w')
f_htr  = open(htr_f, 'w') 

for wid in tr_dict.keys():
    split_idx = len(tr_dict[wid]) // 2
    
    for idx in range(0, split_idx):
        f_gan.write(tr_dict[wid][idx][0] + ' ' + tr_dict[wid][idx][1] + '\n')
    
    for idx in range(split_idx, len(tr_dict[wid])):
        _, img_name = tr_dict[wid][idx][0].split(',')
        f1, f2, _, _ = img_name.split('-')
        img_path = os.path.join(f1, f1 + '-' + f2, img_name + '.png')
        f_htr.write(wid + ',' + img_path + ' ' + tr_dict[wid][idx][1] + '\n')

f_gan.close()
f_htr.close()

The test set is also split in half, however, for the HTR set, it is added to training set that was split and assigned to HTR to obtain a complete dataset.

In [8]:
gan_te_f = "gan.iam.gan_test.filter27"

f_gan_te = open(gan_te_f, 'w')
f_htr  = open(htr_f, 'a') 

for wid in te_dict.keys():
    split_idx = len(te_dict[wid]) // 2

    for idx in range(0, split_idx):
        f_gan_te.write(te_dict[wid][idx][0] + ' ' + te_dict[wid][idx][1] + '\n')

    for idx in range(split_idx, len(te_dict[wid])):
        _, img_name = te_dict[wid][idx][0].split(',')
        f1, f2, _, _ = img_name.split('-')
        img_path = os.path.join(f1, f1 + '-' + f2, img_name + '.png')
        f_htr.write(wid + ',' + img_path + ' ' + te_dict[wid][idx][1] + '\n')

f_gan_te.close()
f_htr.close()

The full HTR dataset has been written, now it is read in again to make the train/val/test splits

In [9]:
with open(htr_f, 'r') as f_htr:
    data_tr = f_htr.readlines()
    data_tr = [i.strip().split(' ') for i in data_tr]
    tr_dict = dict()
    word_len_dict = {}
    word_freq_dict = {}
    words = []
    for i in data_tr:
        wid = i[0].split(',')[0]
        if wid not in tr_dict.keys():
            tr_dict[wid] = [i]
        else:
            tr_dict[wid].append(i)

        if len(i[1]) not in word_len_dict.keys():
            word_len_dict[len(i[1])] = [i[1]]
        else:
            word_len_dict[len(i[1])].append(i[1])

        if i[1] not in word_freq_dict.keys():
            word_freq_dict[i[1]] = [i]
        else:
            word_freq_dict[i[1]].append(i)
        
        words.append(i[1])

print("Number of unique words:", len(np.unique(words)))

for word_len in sorted(word_len_dict.keys()):
    print(word_len, "Number of samples:", len(word_len_dict[word_len]), "Number of unique words:", len(np.unique(word_len_dict[word_len])))

Number of unique words: 4442
2 Number of samples: 6611 Number of unique words: 65
3 Number of samples: 8200 Number of unique words: 279
4 Number of samples: 6395 Number of unique words: 799
5 Number of samples: 4314 Number of unique words: 1042
6 Number of samples: 3260 Number of unique words: 1139
7 Number of samples: 2780 Number of unique words: 1118


In [16]:
oov_htr_te_fn = "htr.iam.oov_test.filter27"
oov_htr_tr_fn = "htr.iam.oov_train.filter27"
oov_htr_val_fn = "htr.iam.oov_val.filter27"

oov_words  = [word for word in  word_freq_dict.keys() if ((len(word_freq_dict[word]) <= 6) and (len(word_freq_dict[word]) >= 3))]

len_oov_words = [len(word) for word in oov_words]
for u_len in np.unique(len_oov_words):
    print(u_len, len_oov_words.count(u_len))

oov_htr_te_f = open(oov_htr_te_fn, 'w')
oov_htr_tr_f = open(oov_htr_tr_fn, 'w')
oov_htr_val_f = open(oov_htr_val_fn, 'w')

for word in oov_words:
    num_samples = len(word_freq_dict[word])
    idx1 = 2 if num_samples > 3 else 1
    idx2 = 4 if num_samples > 4 else 2
    idx3 = num_samples
    print(num_samples, idx1, idx2, idx3)

    for idx in range(0, idx1):
        oov_htr_tr_f.write(word_freq_dict[word][idx][0] + ' ' + word_freq_dict[word][idx][1] + '\n')
    
    for idx in range(idx1, idx2):
        oov_htr_val_f.write(word_freq_dict[word][idx][0] + ' ' + word_freq_dict[word][idx][1] + '\n')

    for idx in range(idx2, idx3):
        oov_htr_te_f.write(word_freq_dict[word][idx][0] + ' ' + word_freq_dict[word][idx][1] + '\n')

oov_htr_te_f.close()
oov_htr_tr_f.close()
oov_htr_val_f.close()

2 10
3 48
4 164
5 211
6 231
7 177
5 2 4 5
6 2 4 6
5 2 4 5
3 1 2 3
4 2 2 4
4 2 2 4
4 2 2 4
5 2 4 5
3 1 2 3
6 2 4 6
6 2 4 6
3 1 2 3
3 1 2 3
3 1 2 3
3 1 2 3
4 2 2 4
3 1 2 3
5 2 4 5
4 2 2 4
3 1 2 3
3 1 2 3
4 2 2 4
5 2 4 5
3 1 2 3
5 2 4 5
3 1 2 3
6 2 4 6
3 1 2 3
3 1 2 3
6 2 4 6
3 1 2 3
5 2 4 5
6 2 4 6
4 2 2 4
4 2 2 4
4 2 2 4
5 2 4 5
3 1 2 3
4 2 2 4
5 2 4 5
6 2 4 6
6 2 4 6
3 1 2 3
4 2 2 4
4 2 2 4
4 2 2 4
4 2 2 4
6 2 4 6
6 2 4 6
5 2 4 5
5 2 4 5
5 2 4 5
5 2 4 5
4 2 2 4
4 2 2 4
4 2 2 4
5 2 4 5
4 2 2 4
3 1 2 3
6 2 4 6
4 2 2 4
5 2 4 5
5 2 4 5
6 2 4 6
3 1 2 3
3 1 2 3
3 1 2 3
5 2 4 5
3 1 2 3
5 2 4 5
3 1 2 3
5 2 4 5
3 1 2 3
6 2 4 6
4 2 2 4
5 2 4 5
4 2 2 4
3 1 2 3
6 2 4 6
4 2 2 4
3 1 2 3
4 2 2 4
4 2 2 4
5 2 4 5
4 2 2 4
3 1 2 3
6 2 4 6
4 2 2 4
3 1 2 3
5 2 4 5
6 2 4 6
3 1 2 3
3 1 2 3
3 1 2 3
4 2 2 4
5 2 4 5
4 2 2 4
3 1 2 3
4 2 2 4
3 1 2 3
3 1 2 3
4 2 2 4
4 2 2 4
3 1 2 3
6 2 4 6
5 2 4 5
3 1 2 3
3 1 2 3
4 2 2 4
4 2 2 4
6 2 4 6
4 2 2 4
3 1 2 3
3 1 2 3
6 2 4 6
4 2 2 4
4 2 2 4
5 2 4 5
6 2 4 6
5 2 4 5
3 1 2 

In [17]:
htr_iv_samples = []
htr_iv_samples_wid = []

total = 0

for wid in tr_dict.keys():
    samples = tr_dict[wid]
    total += len(samples)
    word = [tup[1] for tup in samples]
    htr_iv_samples += [samples[idx] for idx in range(len(samples)) if samples[idx][1] not in oov_words]
    htr_iv_samples_wid += [wid for idx in range(len(samples)) if samples[idx][1] not in oov_words]

print(len(htr_iv_samples_wid), total)


28206 31560


In [18]:
from sklearn.model_selection import train_test_split

htr_val_fn = "htr.iam.val.filter27"
htr_tr_fn = "htr.iam.train.filter27"
htr_te_fn = "htr.iam.test.filter27"

x_train, x_test, wids_train, wids_test = train_test_split(htr_iv_samples, htr_iv_samples_wid, test_size=0.3, random_state=42, stratify=htr_iv_samples_wid, shuffle=True)
print(len(x_train), len(x_test), len(wids_train), len(wids_test))


htr_tr_f = open(htr_tr_fn, 'w')

for tup in x_train:
    htr_tr_f.write(tup[0] + ' ' + tup[1] + '\n')

htr_tr_f.close()

te_dict = {}

for tup in x_test:
    wid, img_path = tup[0].split(',')
    print(wid)

    if wid not in te_dict.keys():
        te_dict[wid] = [tup]
    else:
        te_dict[wid].append(tup)


htr_val_f = open(htr_val_fn, 'w')
htr_te_f = open(htr_te_fn, 'w')

for wid in te_dict.keys():
    split_idx = len(te_dict[wid]) // 2

    for idx in range(0, split_idx):
        htr_val_f.write(te_dict[wid][idx][0] + ' ' + te_dict[wid][idx][1] + '\n')

    for idx in range(split_idx, len(te_dict[wid])):
        htr_te_f.write(te_dict[wid][idx][0] + ' ' + te_dict[wid][idx][1] + '\n')
    
htr_te_f.close()
htr_val_f.close()


19744 8462 19744 8462
206
125
605
123
577
588
663
130
614
181
241
059
634
549
207
262
067
333
106
587
161
061
027
085
591
330
088
143
333
333
150
000
275
073
588
302
040
513
025
225
671
153
563
154
577
205
567
019
210
560
119
582
209
150
118
538
342
336
634
514
017
168
333
037
075
291
153
128
000
000
339
585
241
016
208
634
124
124
582
215
342
000
322
671
178
671
235
293
133
333
242
094
061
207
337
181
152
343
572
250
026
000
094
320
254
008
320
000
644
292
114
000
165
220
000
654
088
295
247
223
332
334
198
338
113
193
274
227
131
018
180
110
126
153
617
665
152
567
287
188
583
134
025
293
587
335
178
152
061
635
153
235
561
586
246
292
108
342
131
028
544
225
248
522
000
671
183
155
026
072
338
118
247
635
555
304
315
285
129
037
000
124
567
630
343
207
212
000
304
528
671
211
288
130
663
588
330
151
221
666
173
644
287
128
325
042
313
152
286
105
610
551
179
197
341
161
021
631
259
588
341
151
287
246
272
164
337
240
095
108
060
044
110
330
062
172
164
159
212
610
333
110
586
588
20

# Old

In [59]:
with open(htr_tr_f, 'r') as f_htr:
    data_tr = f_htr.readlines()
    data_tr = [i.strip().split(' ') for i in data_tr]
    tr_dict = dict()
    for i in data_tr:
        wid = i[0].split(',')[0]
        if wid not in tr_dict.keys():
            htr_iv_dict[wid] = [i]
        else:
            htr_iv_dict[wid].append(i)

htr_te_f = "htr.iam.test.filter27"
htr_tr_f = "htr.iam.train_noval.filter27"

for wid in htr_iv_dict.keys():
    split_idx = len(htr_iv_dict[wid]) // 5

    if split_idx > 0:
        with open(htr_te_f, 'a') as f_htr_te:
            for idx in range(0, split_idx):
                f_htr_te.write(htr_iv_dict[wid][idx][0] + ' ' + htr_iv_dict[wid][idx][1] + '\n')

        with open(htr_tr_f, 'a') as f_htr_tr:
            for idx in range(split_idx, len(htr_iv_dict[wid])):
                f_htr_tr.write(htr_iv_dict[wid][idx][0] + ' ' + htr_iv_dict[wid][idx][1] + '\n')

In [60]:
# te_wids = [key for key in te_dict.keys()]

# random.seed(42)
# wids_te = random.sample(te_wids, len(te_wids) // 2)

# wids_tr = [wid for wid in te_wids if wid not in wids_te]



# for wid in wids_te:
#     with open(gan_te_f, 'a') as f_gan_te:
#         for idx in range(len(te_dict[wid])):
#             f_gan_te.write(te_dict[wid][idx][0] + ' ' + te_dict[wid][idx][1] + '\n')

# for wid in wids_tr:
#     with open(gan_tr_f, 'a') as f_gan_tr:
#         for idx in range(len(te_dict[wid])):
#             f_gan_tr.write(te_dict[wid][idx][0] + ' ' + te_dict[wid][idx][1] + '\n')

In [61]:
with open("htr.iam.tr_va.filter27", 'r') as f_te:
    data_te = f_te.readlines()
    data_te = [i.strip().split(' ') for i in data_te]
    te_dict = dict()
    for i in data_te:
        wid = i[0].split(',')[0]
        if wid not in te_dict.keys():
            te_dict[wid] = [i]
        else:
            te_dict[wid].append(i)

lens = [len(te_dict[wid]) for wid in te_dict.keys()]
print(min(lens))

FileNotFoundError: [Errno 2] No such file or directory: 'htr.iam.tr_va.filter27'

In [ ]:
with open("htr.iam.test.filter27", 'r') as f_te:
    data_te = f_te.readlines()
    data_te = [i.strip().split(' ') for i in data_te]
    te_dict = dict()
    for i in data_te:
        wid = i[0].split(',')[0]
        if wid not in te_dict.keys():
            te_dict[wid] = [i]
        else:
            te_dict[wid].append(i)

lens = [len(te_dict[wid]) for wid in te_dict.keys()]
print(min(lens))

9
